In [ ]:
  
# SKT Brain의 KoBERT, 학습 및 테스트 데이터셋만 따로 준비 = 카카오 브런치[text, label]
# SKT Brain github 주소는 다음과 같습니다. https://github.com/SKTBrain/KoBERT

!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2 # 최신 버전으로 설치하면 "Input: must be Tensor, not str" 라는 에러 발생
!pip install torch

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-nqw9_nop
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-nqw9_nop


In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache() # GPU 캐시 데이터 삭제

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
# GPU 사용 시
#device = torch.device("cuda:0")
device = torch.device("cpu")

# BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [5]:
# 구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# 데이터셋 불러오기
import pandas as pd
data1 = pd.read_excel('/content/drive/My Drive/데캡디/0to499_labeling.xlsx', '합본')

In [7]:
# 불필요한 데이터 column 제거
df = data1[['평균', '리뷰',]]
# column 순서 재배열
df = df[['리뷰', '평균']]
# 데이터 확인
df.head()

,리뷰,평균
0,이브랜드는 올리브영 입점전부터 써본브랜드입니다\n 수분크림구매시에 성분이 중요한데요...,5.0
1,전에 어찌어찌하다 자작나무 수분크림 샘플을 받아 잘 쓰고 있는 중에\n 1+1행사한...,5.0
2,올리브영 대표 클린뷰티! 라운드랩!\n \n # 라운드랩 자작나무 수분 크림 \n ...,4.6
3,💚수분감\n 진짜 내가 찾던 수분만 있는 제품.. 유분은 기름종이 보시면 알겠지만 ...,3.8
4,원쁠원 행사 넘 좋아요 이번달 끝나기 전에 하나 더 사놓을까 고민이 되네요 .. ❄...,4.0


In [8]:
# 평균 반올림
df['평균'] = df['평균'].round()

In [9]:
df.rename(columns={'리뷰':'content', '평균':'label'}, inplace=True)

In [10]:
df = df.astype({'label':'object'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  500 non-null    object
 1   label    500 non-null    object
dtypes: object(2)
memory usage: 7.9+ KB


In [11]:
# label encoding
df.loc[(df['label'] == '1.0'), 'label'] = 0 
df.loc[(df['label'] == '2.0'), 'label'] = 1
df.loc[(df['label'] == '3.0'), 'label'] = 2 
df.loc[(df['label'] == '4.0'), 'label'] = 3 
df.loc[(df['label'] == '5.0'), 'label'] = 4 


# list형태로 데이터 저장 -> [content, label] 형태
data_list = []
for q, label in zip(df['content'], df['label'])  :
    data = []
    data.append(str(q)) # str타입으로 변환, 안 해주면 나중에 타입변환할 때 오류남
    data.append(str(label))

    data_list.append(data)

In [12]:
df = df.astype({'label':'int'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  500 non-null    object
 1   label    500 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 7.9+ KB


In [13]:
df['label_binary'] = [1 if h >= 3 else 0 for h in df['label']]
df['label_binary'].value_counts()

1    299
0    201
Name: label_binary, dtype: int64

In [14]:
# list형태로 데이터 저장 -> [content, label] 형태
data_list = []
for q, label in zip(df['content'], df['label_binary'])  :
    data = []
    data.append(str(q)) # str타입으로 변환, 안 해주면 나중에 타입변환할 때 오류남
    data.append(str(label))

    data_list.append(data)

In [15]:
# 데이터 확인
print(data_list[0])
print(data_list[-1])

['이브랜드는 올리브영 입점전부터 써본브랜드입니다\n 수분크림구매시에 성분이 중요한데요\n 어떤수분크림은 향료나 발림성좋게하기위해 여드름일으키는 쉐어버터 등피이지 피피지 넣는데\n 요즘제품들은 소비자들이 워낙깐칸해서 이런 싸구려 쓰레기들 않넣고 중소기업든 제품들이 성분이 월등히 뛰어납니다\n 브랜드 광고보다 성분보세오ㅡ\n 수부지 ㆍ속건조ㆍ여드름피부추천입니다\n 남성분들도 쓰세요ㅡ추천\n 자작나무수액 몃프로 정확히 기재되어잇고\n 전체성분이 독하고 피부에 해가되는 성분이 없습니다\n 여름에 수분크림필수 쓰셔야 가을되면 피부건조잡티주름 안생깁니다\n 끈적임싫타고 안바르는분 이거바르세오ㅡ\n 가겹도 저렴하고 우리피부를 보호해주는 수분성분이 가든합니다\n 화정전쓰셔도ㅡ밀림없습니다\n 특히 악건성들은 이크림에 페이스오일 두방울 썩어 쓰시면됩니다\n \n 눈밑자글자글고민인분ㅡㅡㅡ꿀팁\n 이크림한번밀어바르고 페이스오일한방울석어서 또바르세오\n 그리고한번더두둘겨바르세요\n 머이렇게 많이 바르냐 하시겟죠\n 주름안생기려면 이렇게 관리하셔야합니ㅏ\n 저는이제품바르고 미백크림또바릅니다\n 그리고 아이크림또한번바릅니다\n 수분이잇어야미백효과가 작용하고 노화도멈춥니다\n 저는50이고10년전부터 올리브영신상들 다사서 성분만골라서 바릅니다\n 주름 안생김ㆍ잡티ㅡ선크림집에서도바름\n 기미없슴\n 자이제답나왓죠\n 수분크림이 제일중요합니다\n 추천', '1']
['처음에는 크림 롤링할때 알갱이가 느껴져서 이게 뭔가 싶었는데 계속 문대니 금새 사라지더라구요 지금같은 여름에 딱인 제품 같아요 확실히 이 크림 바르고 다음날 일어나보면 얼굴 붉은기도 진정된 느낌이구요 쓸수록 좋은 느낌을 받아요 토너랑 같이 짝궁템으로 여름에 쓰기 좋은거 같아요 독도 제품도 그렇구 순하고 자극 없어서 믿고 씁니다 🥰\n ※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별 상품에 따라 용량 내지 일부 구성(1+1, 기획상품 등)이 상이할 수 있음을 안내드립니다.', '1']


In [16]:
# train, test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [17]:
# 데이터 개수 확인
print(len(dataset_train))
print(len(dataset_test))

375
125


In [18]:
dataset_train[0]

['좁쌀 여드름 때문에 아무 제품이나 사용하기 \n 어려웠는데 라운드랩이 워낙 유명하기도 하고 \n 여드름성 피부에도 좋다고 하길래 한 번 속아보는 척하고 구매했는데 이렇게 좋은 걸 왜 안 사고 \n 있었는지 완전 후회했어요ㅠㅠ\n \n 이 제품은 따른 크림들보다 유분감?이 적다고 해야되나 그래서 바를 때 기름진다는 생각이 안 들고 깨끗하게 발리고 흡수도 완전 잘 돼요ㅋㅋㅋ\n \n  바르는 순간 시원한 느낌과 함께 진정효과가 바로 느껴져요!\n \n 안에 작은 알맹이가 있는데 처음에는 뭔가 잘못 \n 들어간건가 했는데 수분 캡슐이라길래 \n 와.. 대박 이러면서 완전 믿고 써요!!ㅋㅋ\n \n 좁쌀은 유수분 밸런스가 중요하다고 하는데 혹시 \n 좁쌀 있으신 분이라면 진짜 완전 강추♡\n ※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별 상품에 따라 용량 내지 일부 구성(1+1, 기획상품 등)이 상이할 수 있음을 안내드립니다.',
 '1']

In [19]:
dataset_train[51]

['제 피부 타입은 수부지에 여드름이 있는 피부인데 요즘같이 매일 마스크를 쓰다 보니 피부가 많이 뒤집어지고 민감해져서 순한 제품을 찾고 있었어요!\n 수분감은 정말 촉촉하고 유분감이 하나도 없어서 너무 좋았어요.\n 또 살짝 쿨링감이 있어서 피부 온도를 살짝 낮춰준다는 느낌도 받았습니당\n 크림 안에 알갱이가 있는데 처음에 바를 때는 알갱이가 피부에 계속 있어서 너무 신경 쓰였는데 손톱으로 살짝 눌러주면 잘 흡수되더라구요!\n 수분감이 많다 보니 아무래도 바르고 나서 살짝 부족하다는 느낌은 있더라구요,,ㅜ\n 봄, 여름에 쓰기 너무 좋을 것 같다는 생각이 듭니당 겨울에는 덧바르거나 추가적으로 다른 크림을 바르면 더 좋을 거 같아요.\n 또 이 크림을 바르고 나서 여드름 개선에 도움이 된다는 느낌은 받지 못했어요,,\n 또 스파츌라를 같이 넣어줘서 그걸로 크림을 덜어서 바르고 있지만 아무래도 위생적이지는 못 한 거 같아요! 용기를 튜브 형식으로 바꿔서 출시하면 어떨까 하는 생각이듭니당! 또 가격이 엄청 착하지는 않았어요, 그래서 행사할 때 많이 챙겨두면 좋을 것 같아용\n ※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별 상품에 따라 용량 내지 일부 구성(1+1, 기획상품 등)이 상이할 수 있음을 안내드립니다.',
 '1']

In [20]:
# KoBERT모델의 입력으로 들어갈 수 있는 형태가 되도록 변환해주는 class
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        # BERTSentenceTransform이라는 모듈 사용
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        # 타입 변환
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    # item 가져오기
    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    # 길이 출력
    def __len__(self):
        return (len(self.labels))

In [21]:
# Setting parameters
max_len = 64
batch_size = 32 # 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [22]:
# 토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

# 패딩
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model


In [23]:
# 데이터 확인 -> array1은 시퀀스, array2는 길이와 타입, array3은 attention mask 시퀀스(1로 패딩된 값들은 연산할 필요가 없기 때문에, 연산을 하지 않아도 됨 -> 이걸 알려주는 데이터가 바로 얘)
data_train[0]

(array([   2,  517, 7267, 6756, 3298, 5920, 6117, 1849, 3111, 4158, 7098,
        2613, 7789, 3220, 6060, 7036, 5761,  517, 6010, 6029, 7096, 3531,
        3585, 7790, 4926, 3298, 5920, 6117, 6573, 4910, 6901, 4206, 5439,
        4924, 5585, 6023, 4955, 2307, 2856, 6797, 6369,  517, 7421, 7788,
        1119, 7867, 3673, 4209,  889, 3466, 3135, 2576, 3871, 5760, 7318,
        3461, 5176, 7953, 7874,    0, 3647, 4158, 7086,    3], dtype=int32),
 array(64, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [24]:
# torch 형식의 dataset 생성
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [25]:
# KoBERT 학습모델
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   # 클래스 수 조정
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [26]:
# BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

# optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio) # 초기에 성능을 끌어올리는 것 = 운동할 때 워밍업 느낌

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# 정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0] # 맞은 비율
    return train_acc
    
train_dataloader

In [ ]:
emb_list = []
logit_list = []
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        emb_list.append(out)
        print(batch_id, out, len(out))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/12 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


0 tensor([[-0.0603, -0.5449],
        [-0.3611, -0.5784],
        [ 0.2100, -0.4474],
        [-0.1847,  0.2710],
        [ 0.0451,  0.0632],
        [-0.0203, -0.4354],
        [-0.0511,  0.1393],
        [-0.2612, -0.1480],
        [ 0.0348, -0.3970],
        [-0.3174, -0.0149],
        [-0.1896, -0.2439],
        [ 0.2590, -0.0397],
        [-0.4839, -0.0034],
        [ 0.3346, -0.1480],
        [ 0.4159, -0.1577],
        [ 0.3145, -0.1387],
        [ 0.3657,  0.1315],
        [ 0.1424, -0.0014],
        [-0.0343,  0.0156],
        [-0.1690,  0.0070],
        [ 0.1363, -0.0919],
        [ 0.0062, -0.2950],
        [-0.1819,  0.1335],
        [-0.2143, -0.7250],
        [-0.1569, -0.3015],
        [-0.2289,  0.1273],
        [ 0.2283,  0.4186],
        [-0.0613,  0.1369],
        [-0.1326,  0.3914],
        [ 0.0040, -0.3868],
        [-0.0562, -0.1167],
        [-0.3100, -0.0630]], grad_fn=<AddmmBackward>) 32
1 tensor([[-0.4657, -0.4181],
        [-0.4986, -0.1868],
        [-0.425

In [ ]:
logit_list = []

# 모델 학습시키기
for e in range(30):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        print(out)
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/188 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


RuntimeError: ignored

In [ ]:
len(emb_list)

122

In [ ]:
emb_1 = []
emb_2 = []
#emb_3 = []
#emb_4 = []
for  i in range(len(emb_list)):
  emb_1.append(emb_list[i][0][0])
  emb_2.append(emb_list[i][0][1])

In [ ]:
a = torch.rand(3,3)
print(a)
b = a.numpy()

tensor([[0.7910, 0.0588, 0.3296],
        [0.1052, 0.8708, 0.0448],
        [0.1020, 0.1415, 0.0279]])


In [ ]:
b

array([[0.07832289, 0.53453255, 0.2271468 ],
       [0.21292388, 0.84869546, 0.6332682 ],
       [0.5696968 , 0.91159123, 0.26018095]], dtype=float32)

In [ ]:
import torch
emb_list

[tensor([[ 0.1443, -0.3982],
         [ 0.4866, -0.5803]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[-0.0117, -0.7728],
         [ 0.9004, -0.1067]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[ 0.2658, -1.1408],
         [-0.0859, -0.0985]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[ 0.4486, -0.9182],
         [ 0.2804, -0.5383]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[ 0.3812, -0.6246],
         [ 0.1093, -0.3311]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[-0.0313, -0.0731],
         [ 0.1785, -0.9972]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[ 0.0699, -0.2303],
         [ 0.3593, -1.0970]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[-0.1475, -0.9112],
         [ 0.3067, -0.6033]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[ 0.4220, -0.6784],
         [ 0.2971, -0.6042]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[ 0.6472, -0.7051],
         [ 0.1856,  0.2129]], device='cuda:0', grad_f

In [ ]:
df_temp['emb_1'] = emb_1
df_temp['emb_2'] = emb_2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df_temp

,content,label,label_binary,emb_1,emb_2
0,이브랜드는 올리브영 입점전부터 써본브랜드입니다\n 수분크림구매시에 성분이 중요한데요...,5,1,"tensor(0.1443, device='cuda:0', grad_fn=<Selec...","tensor(-0.3982, device='cuda:0', grad_fn=<Sele..."
1,전에 어찌어찌하다 자작나무 수분크림 샘플을 받아 잘 쓰고 있는 중에\n 1+1행사한...,5,1,"tensor(-0.0117, device='cuda:0', grad_fn=<Sele...","tensor(-0.7728, device='cuda:0', grad_fn=<Sele..."
2,올리브영 대표 클린뷰티! 라운드랩!\n \n # 라운드랩 자작나무 수분 크림 \n ...,5,1,"tensor(0.2658, device='cuda:0', grad_fn=<Selec...","tensor(-1.1408, device='cuda:0', grad_fn=<Sele..."
3,💚수분감\n 진짜 내가 찾던 수분만 있는 제품.. 유분은 기름종이 보시면 알겠지만 ...,4,1,"tensor(0.4486, device='cuda:0', grad_fn=<Selec...","tensor(-0.9182, device='cuda:0', grad_fn=<Sele..."
4,원쁠원 행사 넘 좋아요 이번달 끝나기 전에 하나 더 사놓을까 고민이 되네요 .. ❄...,4,1,"tensor(0.3812, device='cuda:0', grad_fn=<Selec...","tensor(-0.6246, device='cuda:0', grad_fn=<Sele..."
...,...,...,...,...,...
117,너무 유명한 크림이라 사용해봤어요. 순하고 가벼운 느낌이 마음에 듭니다.,1,0,"tensor(0.3735, device='cuda:0', grad_fn=<Selec...","tensor(-0.4131, device='cuda:0', grad_fn=<Sele..."
118,파워지성이라서 여름에 크림고를때 엄청까다로운데\n 기름지지않고 우선 끈적임ㅇㅣ없어 좋아요,2,0,"tensor(-0.1385, device='cuda:0', grad_fn=<Sele...","tensor(-0.7713, device='cuda:0', grad_fn=<Sele..."
119,"저는 수분크림용으로 구매했다기 보다는,,,,피부과에서 손등부분에 레이져 시술을 했는...",3,1,"tensor(0.6873, device='cuda:0', grad_fn=<Selec...","tensor(-0.8647, device='cuda:0', grad_fn=<Sele..."
120,이 제품이 그렇게 수분크림 강자라고 소문이 났던데 사실인가효..? 또 유튜브보고 영...,4,1,"tensor(0.3014, device='cuda:0', grad_fn=<Selec...","tensor(-0.2426, device='cuda:0', grad_fn=<Sele..."


In [ ]:
df_temp['logit'] = logit_list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_temp['cluster'].value_counts()

2    30
0    29
3    24
1    22
4    17
Name: cluster, dtype: int64

In [ ]:
df_cluster_0 = df_temp[df_temp['cluster'] == 0]
df_cluster_1 = df_temp[df_temp['cluster'] == 1]
df_cluster_2 = df_temp[df_temp['cluster'] == 2]
df_cluster_3 = df_temp[df_temp['cluster'] == 3]
df_cluster_4 = df_temp[df_temp['cluster'] == 4]

In [ ]:
df_cluster_0.

,content,label,label_binary,emb_1,emb_2,cluster,logit
4,원쁠원 행사 넘 좋아요 이번달 끝나기 전에 하나 더 사놓을까 고민이 되네요 .. ❄...,4,1,"tensor(0.3812, device='cuda:0', grad_fn=<Selec...","tensor(-0.6246, device='cuda:0', grad_fn=<Sele...",0,0.298703
8,이건 꽤나 만족스러워 재구매한 수분크림이죠.\n 복합성 피부지만 최근 점점 더 건성...,4,1,"tensor(0.4220, device='cuda:0', grad_fn=<Selec...","tensor(-0.6784, device='cuda:0', grad_fn=<Sele...",0,0.023287
9,최근 라운드랩 제품을 찾으시는 고객님이 많아져 궁금해서 구매해봤어요!!\n 라운드랩...,4,1,"tensor(0.6472, device='cuda:0', grad_fn=<Selec...","tensor(-0.7051, device='cuda:0', grad_fn=<Sele...",0,0.585899
12,라운드랩이 좋다는 소문은 들었으나 괜찮네요:)\n 기존에 사용하던 수분크림을 오랫동...,4,1,"tensor(0.6135, device='cuda:0', grad_fn=<Selec...","tensor(-0.9153, device='cuda:0', grad_fn=<Sele...",0,0.267161
13,지금바르기 좋은 크림이라고해서 구매했어요\n 알갱이가 있는대 손등에바른 사진을 보면...,2,0,"tensor(0.5452, device='cuda:0', grad_fn=<Selec...","tensor(-0.8914, device='cuda:0', grad_fn=<Sele...",0,0.721030
22,안녕하세요 저는 닥터지크림사용하다가 올리브영 매장가서 구경하던중 직원분이 요 상품을...,2,0,"tensor(0.7118, device='cuda:0', grad_fn=<Selec...","tensor(-0.5830, device='cuda:0', grad_fn=<Sele...",0,0.237852
23,여드름성 피부라 특히 여름에는 기름도 많고해서 토너랑 앰플만 발라요. 근데 아무리 ...,3,1,"tensor(0.7101, device='cuda:0', grad_fn=<Selec...","tensor(-0.6697, device='cuda:0', grad_fn=<Sele...",0,0.574958
31,할인하길래 처음으로 사 봤어요!!!!! 평소에 라운드랩 스킨을 잘 써서 구매했는데 ...,2,0,"tensor(0.5280, device='cuda:0', grad_fn=<Selec...","tensor(-0.6657, device='cuda:0', grad_fn=<Sele...",0,0.795803
33,라운드랩 독도 라인은 성분이 순하고 보습이 뛰어나서 이미 수분크림과 선크림을 잘 사...,3,1,"tensor(0.8404, device='cuda:0', grad_fn=<Selec...","tensor(-1.0669, device='cuda:0', grad_fn=<Sele...",0,0.589219
34,자작나무 수분크림이 좋다길래 사 보았어요!\n 저는 지복합성이나 속건조가 있어서 보...,3,1,"tensor(0.6851, device='cuda:0', grad_fn=<Selec...","tensor(-0.7714, device='cuda:0', grad_fn=<Sele...",0,0.778569


In [ ]:
import random

logit_list = []
for i in range(122):
  r = random.random()

  logit_list.append(r)

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5)
y_pred = kmeans.fit_predict(df_temp[['emb_1', 'emb_2']])

In [ ]:
df_temp['cluster'] = y_pred

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_temp

,content,label,label_binary,emb_1,emb_2,cluster
0,이브랜드는 올리브영 입점전부터 써본브랜드입니다\n 수분크림구매시에 성분이 중요한데요...,5,1,"tensor(0.1443, device='cuda:0', grad_fn=<Selec...","tensor(-0.3982, device='cuda:0', grad_fn=<Sele...",1
1,전에 어찌어찌하다 자작나무 수분크림 샘플을 받아 잘 쓰고 있는 중에\n 1+1행사한...,5,1,"tensor(-0.0117, device='cuda:0', grad_fn=<Sele...","tensor(-0.7728, device='cuda:0', grad_fn=<Sele...",2
2,올리브영 대표 클린뷰티! 라운드랩!\n \n # 라운드랩 자작나무 수분 크림 \n ...,5,1,"tensor(0.2658, device='cuda:0', grad_fn=<Selec...","tensor(-1.1408, device='cuda:0', grad_fn=<Sele...",4
3,💚수분감\n 진짜 내가 찾던 수분만 있는 제품.. 유분은 기름종이 보시면 알겠지만 ...,4,1,"tensor(0.4486, device='cuda:0', grad_fn=<Selec...","tensor(-0.9182, device='cuda:0', grad_fn=<Sele...",4
4,원쁠원 행사 넘 좋아요 이번달 끝나기 전에 하나 더 사놓을까 고민이 되네요 .. ❄...,4,1,"tensor(0.3812, device='cuda:0', grad_fn=<Selec...","tensor(-0.6246, device='cuda:0', grad_fn=<Sele...",0
...,...,...,...,...,...,...
117,너무 유명한 크림이라 사용해봤어요. 순하고 가벼운 느낌이 마음에 듭니다.,1,0,"tensor(0.3735, device='cuda:0', grad_fn=<Selec...","tensor(-0.4131, device='cuda:0', grad_fn=<Sele...",1
118,파워지성이라서 여름에 크림고를때 엄청까다로운데\n 기름지지않고 우선 끈적임ㅇㅣ없어 좋아요,2,0,"tensor(-0.1385, device='cuda:0', grad_fn=<Sele...","tensor(-0.7713, device='cuda:0', grad_fn=<Sele...",2
119,"저는 수분크림용으로 구매했다기 보다는,,,,피부과에서 손등부분에 레이져 시술을 했는...",3,1,"tensor(0.6873, device='cuda:0', grad_fn=<Selec...","tensor(-0.8647, device='cuda:0', grad_fn=<Sele...",0
120,이 제품이 그렇게 수분크림 강자라고 소문이 났던데 사실인가효..? 또 유튜브보고 영...,4,1,"tensor(0.3014, device='cuda:0', grad_fn=<Selec...","tensor(-0.2426, device='cuda:0', grad_fn=<Sele...",1


In [ ]:
df_0 = df_cluster_0.sort_values(by='logit' ,ascending=False)
df_1 = df_cluster_1.sort_values(by='logit' ,ascending=False)
df_2 = df_cluster_2.sort_values(by='logit' ,ascending=False)
df_3 = df_cluster_3.sort_values(by='logit' ,ascending=False)
df_4 = df_cluster_4.sort_values(by='logit' ,ascending=False)

In [ ]:
df_0

,content,label,label_binary,emb_1,emb_2,cluster,logit
121,매장직원분께서 추천해주셨는데 넘넘 좋네요~\n 왜 추천인지 알거같아요~\n 촉촉한데...,1,0,"tensor(0.8717, device='cuda:0', grad_fn=<Selec...","tensor(-1.1302, device='cuda:0', grad_fn=<Sele...",0,0.969861
94,헐 원플원이라 바로 삼ㅋㅋ\n 이거 진짜좋아여 ㅋㅋㅋ\n 남친이랑 하나씩 나눠가지려구용,1,0,"tensor(0.5935, device='cuda:0', grad_fn=<Selec...","tensor(-0.6156, device='cuda:0', grad_fn=<Sele...",0,0.889100
31,할인하길래 처음으로 사 봤어요!!!!! 평소에 라운드랩 스킨을 잘 써서 구매했는데 ...,2,0,"tensor(0.5280, device='cuda:0', grad_fn=<Selec...","tensor(-0.6657, device='cuda:0', grad_fn=<Sele...",0,0.795803
34,자작나무 수분크림이 좋다길래 사 보았어요!\n 저는 지복합성이나 속건조가 있어서 보...,3,1,"tensor(0.6851, device='cuda:0', grad_fn=<Selec...","tensor(-0.7714, device='cuda:0', grad_fn=<Sele...",0,0.778569
13,지금바르기 좋은 크림이라고해서 구매했어요\n 알갱이가 있는대 손등에바른 사진을 보면...,2,0,"tensor(0.5452, device='cuda:0', grad_fn=<Selec...","tensor(-0.8914, device='cuda:0', grad_fn=<Sele...",0,0.721030
105,수분크림 쓸만한거 없나 찾고있었는데 이 제품이\n 좋다고 하길래 한번 사봤는데 보습...,4,1,"tensor(0.8012, device='cuda:0', grad_fn=<Selec...","tensor(-0.3936, device='cuda:0', grad_fn=<Sele...",0,0.717997
81,이제 기온도 올라가고 날도 따듯해져서 \n 앞으로 쓰기 좋은 제품을 고르고자 고민하...,4,1,"tensor(0.5094, device='cuda:0', grad_fn=<Selec...","tensor(-0.6850, device='cuda:0', grad_fn=<Sele...",0,0.681327
59,"제 피부 타입은, 민감 복합성 입니다. 그리고 트러블이 있는 피부입니다 ㅠㅠ 근 1...",4,1,"tensor(0.5199, device='cuda:0', grad_fn=<Selec...","tensor(-0.7079, device='cuda:0', grad_fn=<Sele...",0,0.595516
33,라운드랩 독도 라인은 성분이 순하고 보습이 뛰어나서 이미 수분크림과 선크림을 잘 사...,3,1,"tensor(0.8404, device='cuda:0', grad_fn=<Selec...","tensor(-1.0669, device='cuda:0', grad_fn=<Sele...",0,0.589219
9,최근 라운드랩 제품을 찾으시는 고객님이 많아져 궁금해서 구매해봤어요!!\n 라운드랩...,4,1,"tensor(0.6472, device='cuda:0', grad_fn=<Selec...","tensor(-0.7051, device='cuda:0', grad_fn=<Sele...",0,0.585899


In [ ]:
df_1

,content,label,label_binary,emb_1,emb_2,cluster,logit
37,수분크림 다 떨어져서 하나 사려고 하는데..!\n 저는 속 건성에 가까운 만감성 피...,3,1,"tensor(0.3008, device='cuda:0', grad_fn=<Selec...","tensor(-0.3671, device='cuda:0', grad_fn=<Sele...",1,0.993775
35,라운드랩 자작나무 수분크림 샘플사용해보고 너무 좋아서 본품 샀어요. 마침 자작나무 ...,2,0,"tensor(0.5626, device='cuda:0', grad_fn=<Selec...","tensor(0.1074, device='cuda:0', grad_fn=<Selec...",1,0.937024
43,여름에 땀이 난다고해서 수분을 안채워주면 진짜 유분 폭발이예요.... 땀나면 얼굴이...,3,1,"tensor(0.2733, device='cuda:0', grad_fn=<Selec...","tensor(-0.2248, device='cuda:0', grad_fn=<Sele...",1,0.890734
99,피부가 좀 예민한편이라서 순한 크림 찾다가 이 제품 발견한건데..글쎄요ㅠ..생각보다...,2,0,"tensor(0.3205, device='cuda:0', grad_fn=<Selec...","tensor(-0.3147, device='cuda:0', grad_fn=<Sele...",1,0.881419
69,리뷰 합니다 !!\n 이 제품은 제 인생템이라 여러번 사용 후기를 적습니다 저는 크...,3,1,"tensor(0.2881, device='cuda:0', grad_fn=<Selec...","tensor(-0.4194, device='cuda:0', grad_fn=<Sele...",1,0.711342
82,쫀쫀한 워터크림? 제형이에요 수부지라서 수분크림 계속 갈아타는데 괜찮다고 해서 구매...,3,1,"tensor(0.1188, device='cuda:0', grad_fn=<Selec...","tensor(-0.4251, device='cuda:0', grad_fn=<Sele...",1,0.657693
21,가성비 좋고 부담없이 바를수 있던 수분크림이였어요👍🏻\n 홍조있으신 분들 추천드려요...,3,1,"tensor(0.6732, device='cuda:0', grad_fn=<Selec...","tensor(-0.0547, device='cuda:0', grad_fn=<Sele...",1,0.651344
68,자작나무 수분크림 선크림도 로션처럼 흡수되서 진짜정말 좋은데 수분크림도 정말정말 순...,4,1,"tensor(0.1814, device='cuda:0', grad_fn=<Selec...","tensor(-0.4683, device='cuda:0', grad_fn=<Sele...",1,0.641566
64,수분크림 필요해서 사러갔는데 직원추천이더라구요 ㅎ\n 생각보다 너무 촉촉하고 자극이...,2,0,"tensor(0.5727, device='cuda:0', grad_fn=<Selec...","tensor(-0.4429, device='cuda:0', grad_fn=<Sele...",1,0.640290
16,자작나무 수분 선크림을 사용하는데 너무 좋아서 수분크림까지 구매하게 되었습니다😍\n...,3,1,"tensor(0.2584, device='cuda:0', grad_fn=<Selec...","tensor(-0.5618, device='cuda:0', grad_fn=<Sele...",1,0.636013


In [ ]:
df_2

,content,label,label_binary,emb_1,emb_2,cluster,logit
17,수분크림 사야되서 올리브영 보고있었는데 딱 1위에 있더라구요 요즘 독도랑 같이 유명...,3,1,"tensor(-0.1519, device='cuda:0', grad_fn=<Sele...","tensor(-0.8290, device='cuda:0', grad_fn=<Sele...",2,0.980465
58,제 피부 타입은 세안 후 아무것도 바르지 않았을 때 10분만 지나도 유분기가 올라오...,3,1,"tensor(-0.6097, device='cuda:0', grad_fn=<Sele...","tensor(-0.8273, device='cuda:0', grad_fn=<Sele...",2,0.949644
83,"딸아이가 너무 좋아해서 쟁여두고 삽니다. 일단 화해에 보니 알러지, 유해성분이 없어...",3,1,"tensor(0.0455, device='cuda:0', grad_fn=<Selec...","tensor(-0.7143, device='cuda:0', grad_fn=<Sele...",2,0.928509
111,*구매이유*\n 저번에 여름이라 건조해져서 보습이 필요해보이길래 유튜브 찾아보다가 ...,3,1,"tensor(0.0337, device='cuda:0', grad_fn=<Selec...","tensor(-0.8196, device='cuda:0', grad_fn=<Sele...",2,0.923440
106,어떤 피부에서든 부담없이 부드럽게 바르기에 넘 좋아요 밀착력? 발림성?이 조금 떨어...,3,1,"tensor(-0.0512, device='cuda:0', grad_fn=<Sele...","tensor(-0.8688, device='cuda:0', grad_fn=<Sele...",2,0.908151
110,한 번 써본 이후로는 쭉 이 제품만 씁니다. \n 우선 제품이 순해서 한 번도 피부...,2,0,"tensor(0.0370, device='cuda:0', grad_fn=<Selec...","tensor(-0.5622, device='cuda:0', grad_fn=<Sele...",2,0.883264
84,"저는 수부지,복합.민감성인데요\n 순하고 자극없고 피부에 잘 펴발라지네요.\n 수분...",3,1,"tensor(0.2230, device='cuda:0', grad_fn=<Selec...","tensor(-0.7314, device='cuda:0', grad_fn=<Sele...",2,0.755570
103,평소에 토x모리 수분크림 쓰고 자작나무 수분크림은 처음 써봤어용 \n 저랑 엄마랑 ...,3,1,"tensor(0.0122, device='cuda:0', grad_fn=<Selec...","tensor(-0.4713, device='cuda:0', grad_fn=<Sele...",2,0.730897
104,"일단 제 피부타입은 복합성, 민감성, 여드름 피부에요\n 피지오겔 레드 에이아이 크...",3,1,"tensor(0.0440, device='cuda:0', grad_fn=<Selec...","tensor(-0.6296, device='cuda:0', grad_fn=<Sele...",2,0.720101
7,🥝제품정보\n 냄새: 막코인 저에게는 아무냄새도 나지 않았어요\n \n 제형: 딱 ...,3,1,"tensor(-0.1475, device='cuda:0', grad_fn=<Sele...","tensor(-0.9112, device='cuda:0', grad_fn=<Sele...",2,0.662622


In [ ]:
df_3

,content,label,label_binary,emb_1,emb_2,cluster,logit
41,✋수분 크림 치고는 상당히 가벼워요.\n \n ✋마무리감이 가볍게 끝나고 끈적임 없...,3,1,"tensor(0.1250, device='cuda:0', grad_fn=<Selec...","tensor(-0.0277, device='cuda:0', grad_fn=<Sele...",3,0.974835
40,저는 원래 닥터디 레드 블레미쉬 수딩크림을 사용하고 있었습니다 그 제품을 다 쓴 거...,4,1,"tensor(0.0133, device='cuda:0', grad_fn=<Selec...","tensor(-0.3530, device='cuda:0', grad_fn=<Sele...",3,0.924359
115,저번에 두통째비우고 잠시 다른걸로 쓰다가 이번에 다시 재구매했어용 원쁠원 행사중이라...,3,1,"tensor(-0.1651, device='cuda:0', grad_fn=<Sele...","tensor(0.1143, device='cuda:0', grad_fn=<Selec...",3,0.901169
28,"화해 어플에서 구매하서 사용하다가 탑리뷰어 쿠폰 받아서 올영에서 재구매 했어요,\n...",4,1,"tensor(-0.4039, device='cuda:0', grad_fn=<Sele...","tensor(-0.1180, device='cuda:0', grad_fn=<Sele...",3,0.882717
6,진짜 되게 순해요....ㅎㅎ....이거 전에 올리브영에서 그이벤트 당첨되서 받고 써...,4,1,"tensor(0.0699, device='cuda:0', grad_fn=<Selec...","tensor(-0.2303, device='cuda:0', grad_fn=<Sele...",3,0.790063
36,🥰이번에 재구매 했어요!!🥰\n \n 이 제품의 단점부터 말하자면 자작나무 크림 안...,3,1,"tensor(0.1434, device='cuda:0', grad_fn=<Selec...","tensor(-0.1950, device='cuda:0', grad_fn=<Sele...",3,0.724100
39,많이 발라서 그런가 좀 꾸덕하게 발리지만 끈적함 있는 꾸덕함이 아니고 흡수되면 촉촉...,2,0,"tensor(-0.1663, device='cuda:0', grad_fn=<Sele...","tensor(-0.2357, device='cuda:0', grad_fn=<Sele...",3,0.673913
90,크림은 이것저것 정말 많이 썼는데 이 제품이 촉촉하면서 흡수도 잘되고 좋네요,1,0,"tensor(0.1227, device='cuda:0', grad_fn=<Selec...","tensor(-0.0998, device='cuda:0', grad_fn=<Sele...",3,0.672360
26,제가 처음 자작나무를 접했을때가 생각나네요👍\n 원래 그림이 귀엽고 이미 독도라인으...,2,0,"tensor(-0.1566, device='cuda:0', grad_fn=<Sele...","tensor(-0.1162, device='cuda:0', grad_fn=<Sele...",3,0.602977
45,라운드랩은 워낙 독도때문에 유명하니깐 불안감없이 사봤어여 평소 전 그날 그날 피부상...,4,1,"tensor(-0.2045, device='cuda:0', grad_fn=<Sele...","tensor(-0.3803, device='cuda:0', grad_fn=<Sele...",3,0.499237


In [ ]:
df_4

,content,label,label_binary,emb_1,emb_2,cluster,logit
112,*구매이유*\n 저는 리프팅크림을 사서 스킨로션리프팅크림만 발라주고 있었어요. 부족...,3,1,"tensor(0.3372, device='cuda:0', grad_fn=<Selec...","tensor(-1.1977, device='cuda:0', grad_fn=<Sele...",4,0.907612
78,올리부영에서 늘 구매하던 수분크림이 있어요!! \n 진짜 수분감이 많고 진정에 좋아...,3,1,"tensor(0.2365, device='cuda:0', grad_fn=<Selec...","tensor(-0.9430, device='cuda:0', grad_fn=<Sele...",4,0.907507
54,✔ 보습력\n 끈적임없이 촉촉해요. 겉도는 느낌없이 속보습도 잘 채워주는 것 같아요...,3,1,"tensor(0.2109, device='cuda:0', grad_fn=<Selec...","tensor(-1.1673, device='cuda:0', grad_fn=<Sele...",4,0.839610
86,제 피부 타입은 일단 티존 나비존엔 기름기가 뿜어져나오는 지성이고 이마 볼쪽은 건조...,4,1,"tensor(0.3804, device='cuda:0', grad_fn=<Selec...","tensor(-1.1393, device='cuda:0', grad_fn=<Sele...",4,0.815195
56,성분이 좋아보이길래 구매했습니다. 복합성 수부지인 제 피부에는 맞지 않았지만 건성 ...,3,1,"tensor(0.3814, device='cuda:0', grad_fn=<Selec...","tensor(-1.3488, device='cuda:0', grad_fn=<Sele...",4,0.810109
95,더운데 출퇴근 시 대중교통 이용 시간이 길고 직업 특성상 사람 만나는 일이 많아서 ...,3,1,"tensor(0.6258, device='cuda:0', grad_fn=<Selec...","tensor(-1.2875, device='cuda:0', grad_fn=<Sele...",4,0.802295
76,보습이 좋은 것 같아요\n 라운드랩 꺼 수분선크림 발라봤는데\n 순하고 자극적이지 ...,2,0,"tensor(0.3211, device='cuda:0', grad_fn=<Selec...","tensor(-0.9649, device='cuda:0', grad_fn=<Sele...",4,0.761239
51,"요즘 더운데 마스크 하루종일 쓰느라 열이 얼굴에 갇혀서 피지며 여드름이 진짜 폭발,...",2,0,"tensor(0.5611, device='cuda:0', grad_fn=<Selec...","tensor(-1.0748, device='cuda:0', grad_fn=<Sele...",4,0.736059
48,수분크림이 다 떨어져가는 상황에서 급한대로 오프라인매장에서 구매했어요. 온라인몰에서...,2,0,"tensor(0.3714, device='cuda:0', grad_fn=<Selec...","tensor(-1.1512, device='cuda:0', grad_fn=<Sele...",4,0.631347
67,피부가예민한편인데지복합성피부에\n 마스크도많이쓰고트러블에건조해서\n 순한수분크림찾고...,3,1,"tensor(0.1966, device='cuda:0', grad_fn=<Selec...","tensor(-1.0275, device='cuda:0', grad_fn=<Sele...",4,0.616926


In [ ]:
df_0.iloc[0]['content']

'매장직원분께서 추천해주셨는데 넘넘 좋네요~\n 왜 추천인지 알거같아요~\n 촉촉한데 잘흡수되고 끈적임은 없어요~'

In [ ]:
df_0.iloc[1]['content']

'헐 원플원이라 바로 삼ㅋㅋ\n 이거 진짜좋아여 ㅋㅋㅋ\n 남친이랑 하나씩 나눠가지려구용'

In [ ]:
df_0.iloc[2]['content']

'할인하길래 처음으로 사 봤어요!!!!! 평소에 라운드랩 스킨을 잘 써서 구매했는데 보습력은 정말 좋은 것 같습니다…..!!! 다만 크림 안에 겔 같은거 좀 거슬러요 ㅠㅠㅠㅠㅠㅠ 하나하나 일일이 다 터트려야 해요…..ㅠㅠㅠㅠㅠ 안그러면 이게 때처럼 밀리더라구요 ㅠㅠ 다른 겔들은 시간이 지나면 알아서 녹거나 흡수되는데 이 알들은 양도 많은데 흡수가 안 돼요…… 아침에 바쁠 땐 하나하나 터트리기 귀찮습니다 ㅠㅠㅠㅠㅠ 그래도 올리브영에서 프로모션이랑 세일해서 싸게 구매해서 손해본 느낌은 나지 않내요!!!!! 사용 후 부작용이나 트러블도 딱히 올라오지 않구요 ㅠㅠㅠㅠㅠ 겔 흡수만 개선되면 훨씬 좋을 것 같습니다 ㅎㅎㅎㅎㅎ 가성비는 정말 좋은 상품이지만 겔 때문에 또 구매하지는 않을 것 같습니다 ㅠㅠㅠㅠ'

In [ ]:
df_0.iloc[3]['content']

'자작나무 수분크림이 좋다길래 사 보았어요!\n 저는 지복합성이나 속건조가 있어서 보습+ 수분감 기능이 아주 중요한데요. \n \n 자작나무 수분크림은 디렉터 파이가 추천하기도 했고 속건조에 좋다고 해서 구매했어요 ㅎㅎ\n \n 순한성분과 뛰어난 보습력을 갖고있으나\n 여름에 쓰기에는 살짝 무거울 수 있을 것 같아요 ㅎㅎ\n \n 완전 극건성이나 여름에도 무겁고\n 강력한 수분크림을 원한다면 추천드리지만 ...\n 꾸덕하고 기능이 너무 좋은 수분크림이기 때문에 ㅋㅋ\n 지복합성에게는 !!!\n 가을 겨울 봄에 추천드립니다 !! \n \n 일단 쟁여두고 찬바람 불때 저는 써야겠아녀 ㅎ\n ※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별 상품에 따라 용량 내지 일부 구성(1+1, 기획상품 등)이 상이할 수 있음을 안내드립니다.'

In [ ]:
df_0.iloc[4]['content']

'지금바르기 좋은 크림이라고해서 구매했어요\n 알갱이가 있는대 손등에바른 사진을 보면 \n 알갱이가 살아있는대\n 얼굴에 바르니 흡수 잘되요\n 가볍고 흡수도 좋아서 지금 바르기좋아요\n 2번정도 덧발라도 무거움 없습니다\n 패드는 외출용으로 좋아요\n 겨울에 바르기엔 수분 부족일듯합니다'